In [ ]:
import requests
from requests.auth import HTTPBasicAuth

class APIClient:
    def __init__(self, token_url, client_id, client_secret, api_base_url):
        self.token_url = token_url
        self.client_id = client_id
        self.client_secret = client_secret
        self.api_base_url = api_base_url
        self.session = requests.Session()
        self._get_token()

    def _get_token(self):
        data = {
            'grant_type': 'client_credentials',
        }
        auth = HTTPBasicAuth(self.client_id, self.client_secret)
        response = self.session.post(self.token_url, data=data, auth=auth)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
        token = response.json()['access_token']
        self.session.headers.update({'Authorization': f'Bearer {token}'})
        
    def get_resource(self, endpoint):
        url = f'{self.api_base_url}/{endpoint}'
        response = self.session.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
        return response.json()

# Usage:
client = APIClient(
    token_url='https://authorization-server.com/token',
    client_id='your-client-id',
    client_secret='your-client-secret',
    api_base_url='https://api.example.com'
)

resource_data = client.get_resource('protected-resource')
print(resource_data)


In [17]:
import pandas as pd
import json

# Sample deeply nested JSON data (as string)
deep_resource_data_str = """
{
    "people": [
        {
            "name": "Alice", 
            "age": 25, 
            "addresses": [
                {"details": {"city": "New York", "state": "NY", "extra": {"zip": "10001", "country": "USA"}}},
                {"details": {"city": "Boston", "state": "MA", "extra": {"zip": "02101", "country": "USA"}}}
            ]
        },
        {
            "name": "Bob", 
            "age": 30, 
            "addresses": [
                {"details": {"city": "Los Angeles", "state": "CA", "extra": {"zip": "90001", "country": "USA"}}},
                {"details": {"city": "San Francisco", "state": "CA", "extra": {"zip": "94101", "country": "USA"}}}
            ]
        }
    ]
}
"""

# Convert JSON string to a dictionary
deep_resource_data = json.loads(deep_resource_data_str)

# Normalize the 'addresses' level
df_addresses = pd.json_normalize(deep_resource_data['people'], record_path='addresses', meta=['name', 'age'])

df_addresses.head(3)

,details.city,details.state,details.extra.zip,details.extra.country,name,age
0,New York,NY,10001,USA,Alice,25
1,Boston,MA,02101,USA,Alice,25
2,Los Angeles,CA,90001,USA,Bob,30


In [16]:
# Construct the final flattened DataFrame
df_final = df_addresses[['name', 'age', 'details.city', 'details.state', 'details.extra.zip', 'details.extra.country']]
df_final.columns = ['name', 'age', 'city', 'state', 'zip', 'country']

df_final



,name,age,city,state,zip,country
0,Alice,25,New York,NY,10001,USA
1,Alice,25,Boston,MA,02101,USA
2,Bob,30,Los Angeles,CA,90001,USA
